In [276]:
from sklearn.datasets import load_digits
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [277]:
data=load_digits()

In [278]:
data['target_names']

array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])

In [279]:
X=pd.DataFrame(data["data"])

In [280]:
Y=pd.Series(data["target"])

In [281]:
Y_b=label_binarize(Y,classes=list(range(10)))

In [282]:
X_test, X_train, y_test, y_train=train_test_split(X, Y, test_size=0.2, random_state=666 )

In [316]:
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import SGDClassifier
from sklearn.svm import SVC

##### Набираем модели: возьмём, логРегрессию, Деревья ,Соседей, ГрадСпуск

In [319]:
lr=LogisticRegression
dtc=DecisionTreeClassifier
knn=KNeighborsClassifier
sgd=SGDClassifier
svm=SVC

In [333]:
models={'lr':LogisticRegression,'dtc':DecisionTreeClassifier, 'knn':KNeighborsClassifier, 'sgd': SGDClassifier,'svm':SVC}

##### Набираем параметры для моделей

In [358]:
c_svm=np.linspace(0.1,1, num=20)
tol=10**np.linspace(-4,1, num=5)
grid5={
      'C':c_svm
    , 'kernel':['linear', 'poly', 'rbf', 'sigmoid']
    , 'tol': tol           
               }
gs5=GridSearchCV(svm(), grid5, scoring='accuracy', cv=5)
gs5.fit(X_train,y_train)
gs5.best_params_
#model5=svm(**gs5.best_params_)
#model5.fit(X_train,y_train)
#pred5=model5.predict(X_test)
#accuracy_score(y_test,pred5)

{'C': 0.10000000000000001, 'kernel': 'linear', 'tol': 0.56234132519034907}

In [337]:
l1r=np.linspace(0,1,num=20)
grid4={
     'penalty':['none','l2', 'l1','elasticnet']
    ,'loss':['hinge', 'log', 'modified_huber', 'squared_hinge', 'perceptron','squared_loss', 'huber', 'epsilon_insensitive','squared_epsilon_insensitive']
    ,'l1_ratio':l1r}
gs4=GridSearchCV(sgd(), grid4, scoring='accuracy', cv=5)
gs4.fit(X_train,y_train)
gs4.best_params_
#gs4.fit(X_train, y_train)
#pred4=gs4.predict(X_test)
#accuracy_score(y_test,pred4)

{'l1_ratio': 0.21052631578947367, 'loss': 'modified_huber', 'penalty': 'none'}

In [287]:
cs=10**np.linspace(-3,1,5)
grid1={"C":cs, "penalty":['l1','l2'],'solver':['liblinear'], 'dual':[bool(0)]}
gs1=GridSearchCV(lr(),grid1,scoring='accuracy', cv=5)
gs1.fit(X_train,y_train)
gs1.best_params_
#model1=lr(**gs1.best_params_)
#model1.fit(X_train,y_train)
#pred1=model1.predict(X_test)
#accuracy_score(y_test,pred1)

{'C': 0.01, 'dual': False, 'penalty': 'l2', 'solver': 'liblinear'}

In [288]:
grid2={
      'max_depth':list(range(1,20))
    , 'criterion':['gini','entropy']
    , 'splitter':['best','random']
    , 'class_weight':['balanced',None]}
gs2=GridSearchCV(DecisionTreeClassifier(), grid2, scoring='accuracy')
gs2.fit(X_train, y_train)
gs2.best_params_

{'class_weight': None,
 'criterion': 'entropy',
 'max_depth': 13,
 'splitter': 'best'}

In [289]:
grid3={'n_neighbors':list(range(1,6)), 'algorithm':['auto', 'ball_tree', 'kd_tree', 'brute'], 'weights':['uniform','distance'], 'p':[1,2]}
gs3=GridSearchCV(knn(),grid3, scoring='accuracy')
gs3.fit(X_train,y_train)
gs3.best_params_
#model3=knn(**gs3.best_params_)
#model3.fit(X_train,y_train)
#pred3=model3.predict(X_test)
#accuracy_score(y_test, pred3)

{'algorithm': 'auto', 'n_neighbors': 1, 'p': 2, 'weights': 'uniform'}

##### Запихиваем в цикл

In [360]:
score=[]
for x in models.values():
    if x==lr:
        grids=grid1
    elif x==dtc:
        grids=grid2
    elif x==knn:
        grids=grid3
    elif x==sgd:
        grids=grid4
    elif x==svm:
        grids=grid5
    gs=GridSearchCV(x(), grids, scoring='accuracy')
    gs.fit(X_train,y_train)
    model=x(**gs.best_params_)
    model.fit(X_train,y_train)
    pred=model.predict(X_test)
    score.append(accuracy_score(y_test,pred ))
print(*(zip(score, models.keys())))

(0.94084899095337504, 'lr') (0.76548364648573419, 'dtc') (0.96520528879610301, 'knn') (0.91579679888656929, 'sgd') (0.96520528879610301, 'svm')


Получаем, что лучшая модель у нас **KNN** и **SMV**, причём скор у них одинаковый.